In [1]:
import h2o
from h2o.estimators.coxph import H2OCoxProportionalHazardsEstimator
h2o.init()


Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,21 mins 11 secs
H2O_cluster_timezone:,America/Santiago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.2
H2O_cluster_version_age:,1 month and 15 days
H2O_cluster_name:,H2O_from_python_mwils_47jp78
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.261 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [4]:
# Import the heart dataset into H2O:
heart = h2o.import_file("http://s3.amazonaws.com/h2o-public-test-data/smalldata/coxph_test/heart.csv")
print(heart.head())

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


start,stop,event,age,year,surgery,transplant,id
0,50,1,-17.1554,0.123203,0,0,1
0,6,1,3.83573,0.25462,0,0,2
0,1,0,6.29706,0.265572,0,0,3
1,16,1,6.29706,0.265572,0,1,3
0,36,0,-7.73717,0.490075,0,0,4
36,39,1,-7.73717,0.490075,0,1,4
0,18,1,-27.2142,0.607803,0,0,5
0,3,1,6.59548,0.70089,0,0,6
0,51,0,2.86927,0.780287,0,0,7
51,675,1,2.86927,0.780287,0,1,7


In [5]:
# Split the dataset into a train and test set:
train, test = heart.split_frame(ratios = [.8], seed = 1234)

In [6]:
# Build and train the model:
heart_coxph = H2OCoxProportionalHazardsEstimator(start_column="start",
                                                 stop_column="stop",
                                                 ties="breslow")
heart_coxph.train(x="age",
            y="event",
            training_frame=train)

c:\Users\mwils\Documents\Python\ML\venv\lib\site-packages\h2o\estimators\estimator_base.py:208: RuntimeWarning: Response is numeric, so the regression model will be trained. However, the cardinality is equaled to two, so if you want to train a classification model, convert the response column to categorical before training.
  warnings.warn(mesg["message"], RuntimeWarning)


coxph Model Build progress: |████████████████████████████████████████████████████| (done) 100%
Model Details
H2OCoxProportionalHazardsEstimator :  Cox Proportional Hazards
Model Key:  CoxPH_model_python_1657564349268_1

Call: 
Surv(start, stop, event) ~ age

Coefficients: CoxPH Coefficients


,names,coefficients,exp_coef,exp_neg_coef,se_coef,z_coef
0,age,0.022568,1.022824,0.977685,0.015144,1.490247



Likelihood ratio test=2.422254
Concordance=0.557352
n=138, number of events=59

ModelMetricsRegressionCoxPH: coxph
** Reported on train data. **

MSE: NaN
RMSE: NaN
Concordance score: 0.5573518090839107
Concordant count: 2169
Tied cout: 6

Scoring History: 


,,timestamp,duration,iterations,loglik
0,,2022-07-11 14:56:35,0.000 sec,0,-220.069362
1,,2022-07-11 14:56:35,0.002 sec,1,-218.874684
2,,2022-07-11 14:56:35,0.004 sec,2,-218.858239
3,,2022-07-11 14:56:35,0.005 sec,3,-218.858235
4,,2022-07-11 14:56:35,0.006 sec,4,-218.858235


In [8]:
# Generate predictions on a test set (if necessary):
pred = heart_coxph.predict(test)
print(pred.head())

coxph prediction progress: |█████████████████████████████████████████████████████| (done) 100%


lp
0.209418
0.132061
0.0668752
0.215659
0.215659
0.202992
0.202992
0.198544
0.267004
0.2316


In [10]:
# Get baseline hazard:
hazard = heart_coxph.baseline_hazard_frame
print(hazard)

t,baseline hazard
1,0
2,0.0143776
3,0.0224131
4,0
5,0.00787185
6,0.00806003
8,0.00821637
9,0.00844234
10,0
11,0


In [11]:
survival = heart_coxph.baseline_survival_frame
print(survival)

t,baseline survival
1,1
2,0.985725
3,0.963878
4,0.963878
5,0.95632
6,0.948643
8,0.940881
9,0.932971
10,0.932971
11,0.932971


In [12]:
heart_coxph.model_performance().concordance()

0.5573518090839107